In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data Ingestion

In [18]:
path = "data/"
bixi_dat = pd.read_csv(path+"bixi_wrt_cal_15min.csv", parse_dates = ['index'],  index_col=0)
bixi_dat.rename(columns={'index':'timestamp'}, inplace=True)
bixi_dat

,timestamp,com1,com2,com3,com4,com5,com6,business_day,tod_afternoon,tod_afternoon_rush,tod_evening,tod_lunch_time,tod_morning,tod_morning_rush,tod_night,Temperature,Precipitation,Wind Speed,Relative Humidity
0,2017-04-15 00:00:00,8.0,5.0,4.0,11.0,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
1,2017-04-15 00:15:00,1.0,5.0,3.0,8.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
2,2017-04-15 00:30:00,2.0,5.0,6.0,9.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
3,2017-04-15 00:45:00,3.0,3.0,4.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
4,2017-04-15 01:00:00,7.0,4.0,3.0,6.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.3,0.0,7.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81137,2021-05-31 22:00:00,29.0,18.0,33.0,30.0,77.0,22.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81138,2021-05-31 22:15:00,11.0,9.0,15.0,15.0,40.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81139,2021-05-31 22:30:00,21.0,32.0,33.0,24.0,60.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81140,2021-05-31 22:45:00,21.0,29.0,27.0,39.0,55.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0


In [43]:
# Bike-sharing demand for each community 
demand_features = ['com1', 'com2', 'com3', 'com4', 'com5', 'com6']
# Weather-calender variables
extra_features = ['business_day', 'tod_afternoon', 'tod_afternoon_rush','tod_evening','tod_lunch_time','tod_morning','tod_morning_rush','tod_night','Temperature','Precipitation', 'Wind Speed', 'Relative Humidity']                        

# Bike sharing demand for each community over time
bixi_demand = bixi_dat[demand_features].values
print(demand_dat.shape)
# Weather-calender variables over time
weather_calender = bixi_dat[extra_features].values
print(weather_calender.shape)

(81309, 6)
(81309, 12)


### Data Preperation

In [44]:

def split_sequence(sequence, lag):
    '''
    This function splits a given univariate sequence into
    multiple samples where each sample has a specified number 
    of time steps and the output is a single time step.
    '''
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + lag

    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def convert_to_supervised(lag, dat1, dat2=None):
    '''    
    This function takes a 2D sequennce, scales the array and splits 
    a given multivariate sequence into multiple samples where each sample has a specified number 
    of time steps. It returns multiple time steps as well as the scaler.
    param dat1: Bike sharing demand for each community over time
    param dat2: Weather-calender variables over time
    '''
    _, start_test = train_val_test()
    if dat2 is None:
        # scale data to [-1, 1]
        scaler = MinMaxScaler(feature_range=(-1, 1))
        dat1_train, dat1_test = train_test_split(X=dat1, start_test = start_test)
        scaled_train = scaler.fit_transform(dat1_train)
        scaled_test = scaler.transform(dat1_test)
        scaled_dat = np.vstack([scaled_train, scaled_test])
        m, n = scaled_dat.shape
        # e.g., if lag = 7, BIXI demand of past 7*15 minutes
        scaled_X = np.zeros((m-lag,lag, n))
        scaled_y = np.zeros((m-lag,n))
        
        for i in range(0,n):
            X, y = split_sequence(scaled_dat[:,i],lag)
            scaled_X[:,:,i] = X
            scaled_y[:,i] = y
        return scaler, scaled_X, scaled_y
    
    else:        
        scaler1 = MinMaxScaler(feature_range=(-1, 1))        
        dat1_train, dat1_test = train_test_split(X=dat1, start_test = start_test)        
        scaled_train1 = scaler1.fit_transform(dat1_train)
        scaled_test1 = scaler1.transform(dat1_test)
        
        scaler2 = MinMaxScaler(feature_range=(-1, 1))
        dat2_train, dat2_test = train_test_split(X=dat2, start_test = start_test)
        scaled_train2 = scaler2.fit_transform(dat2_train)
        scaled_test2 = scaler2.transform(dat2_test)

        scaled_dat1 = np.vstack([scaled_train1, scaled_test1])
        scaled_dat2 = np.vstack([scaled_train2, scaled_test2])

        m, n = scaled_dat1.shape
        _, k = scaled_dat2.shape

        # Bixi demand of the past lag days
        scaled_X = np.zeros((m-lag, lag+k, n))
        scaled_y = np.zeros((m-lag,n))

        for i in range(0,n):
            X,y = split_sequence(scaled_dat1[:,i],lag)
            X = np.hstack([X,scaled_dat2[lag:,:]])
            scaled_X[:,:,i] = X
            scaled_y[:,i] = y
        return scaler1, scaler2, scaled_X, scaled_y

def invert_scale(scaler, y):
    '''
    Inverse scaling for a predicted value
    '''
    return scaler.inverse_transform(y)

def train_val_test(dat=bixi_dat, train_end_date='2020-04-15 10:45:00', val_end_date='2020-07-15 18:15:00'):
    '''
    Important: determine train, test and validation based on Covid outbreak
    param train_end (timstamp): end of training set
    param val_end (timestamp): end of validation set
    '''
    assert train_end_date < val_end_date, "WARNING: validation date must be later than training date."
    train = dat.timestamp[dat.timestamp == train_end_date].index.values[0]   
    validation = dat.timestamp[dat.timestamp == val_end_date].index.values[0]   
    train_val_ratio = validation/train   
    # test set preparation    
    start_test =  len(dat) - validation

    print(f"Train set starts at {dat.loc[0,'timestamp']} and ends at {dat.loc[train,'timestamp']}")
    print(f"Validation set starts at {dat.loc[train,'timestamp']} and ends at {dat.loc[validation,'timestamp']}")
    print(f"Test set starts at {dat.loc[validation,'timestamp']} and ends at {dat.loc[len(dat),'timestamp']}")
    return train_val_ratio, start_test

def train_test_split(X, y = None, start_test = None, ratio= 0.3):
    '''
    Train test split for times-series data
    '''
    if start_test is None:
        lenght,_ = X.shape
        start_test = int(ratio*lenght)

    X_train, X_test = X[0:-start_test], X[-start_test:] 
    print("X training size: ", X_train.shape)
    print("X test size: ",X_test.shape )
    if y is None:
        return  X_train, X_test  
    
    else:
        y_train, y_test = y[:-start_test], y[-start_test:]
        return X_train, X_test, y_train, y_test

def model_dim(X):
    """
    Deep learning architucture size for input, fearures and output
    """
    n_features, n_steps_out = X.shape[2], X.shape[2]
    n_steps_in = X.shape[1]
    return n_steps_in, n_features, n_steps_out


def integer_factor(n):
    """ calculate integer factorization of n
    and then returns them as two multiplications"""
    def is_prime(n):
        if n == 1:
            return False
        if n % 2 == 0:
            return False
        i = 3
        while i * i <= n:
            if n % i == 0:
                return False
            i += 2
        return True

    def prime_factors(n):
        prime_factor_list = []
        prime_factor_list.append(1)
        for i in itertools.chain([2], itertools.count(3, 2)):
            if n <= 1:
                break
            while n % i == 0:
                n //= i
                prime_factor_list.append(i)
        return prime_factor_list

    lst = prime_factors(n)
    lng = len(lst)
    half= int(np.round(lng/2+1))
    list1, list2 = lst[0:half], lst[half:]
    n_steps, n_sequence = np.prod(list1), np.prod(list2)
    return n_steps, n_sequence


def demand_extra_feature(dat):
    '''
    This function adds engineered features to demand dataset.
    '''    
    Mean = dat.mean(1)
    Max = dat.max(1)
    Min = dat.min(1)
    Q25 = np.quantile(dat,0.25,axis=1)
    Q50 = np.quantile(dat,0.50,axis=1)
    Q75 = np.quantile(dat,0.75,axis=1)
    Std = dat.std(axis=1)
    return np.vstack([Mean,Max,Min,Q25,Q50,Q75,Std]).T

### Deep Learning Models Archituctures

In [ ]:
## LSTM family
def one_LSTM(n_steps_in, n_features, n_steps_out, lstm_size, dropout, dc_size):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(n_steps_in, n_features)))
    model.add(LSTM(lstm_size, dropout=dropout, recurrent_dropout=0, activation='tanh', return_sequences=True))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(dc_size))
    model.add(Dropout(dropout))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['MSE'])
    return model

def one_biLSTM(n_steps_in, n_features, n_steps_out, lstm_size, dropout, dc_size):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(n_steps_in, n_features)))
    model.add(Bidirectional(LSTM(lstm_size, dropout=dropout, recurrent_dropout=0, activation='tanh', return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(dc_size))
    model.add(Dropout(dropout))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['MSE'])
    return model

## CNN-LSTM family
def TreNet_LSTM(n_steps_in, n_features, n_steps_out, filters, lstm_size, dropout, dc_size):
    n_steps, n_seq =  integer_factor(n_steps_in)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1),
                          input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(lstm_size, activation='tanh', recurrent_dropout=0))
    model.add(Dense(dc_size))
    model.add(Dropout(dropout))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['MSE'])
    return model

def TreNet_biLSTM(n_steps_in, n_features, n_steps_out, filters, lstm_size, dropout, dc_size):
    n_steps, n_seq =  integer_factor(n_steps_in)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1),
                          input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(lstm_size, dropout=dropout, recurrent_dropout=0, activation='tanh')))
    model.add(Dense(dc_size))
    model.add(Dropout(dropout))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['MSE'])
    return model

### Visualization

In [41]:
def model_loss(history,label2='Validation Loss'):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label=label2)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.grid(linestyle="--")
    plt.show();

def evaluate_forecasts(actual, predicted, text = "Test", plot=True):
    """
    Evaluate prediction performance based on RMSE and MAE
    """
    RMSEs = list()
    MAEs = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = np.sqrt(mse)
        # store
        RMSEs.append(rmse)

        # calculate mae
        mae = mean_absolute_error(actual[:,i], predicted[:,i])
        # store
        MAEs.append(mae)

    # calculate overall RMSE and MAE
    y_true = actual.flatten()
    y_hat = predicted.flatten()

    overal_mae = mean_absolute_error(y_true, y_hat)
    overal_rmse = np.sqrt(mean_squared_error(y_true, y_hat))

    print("#### Evaluating performance metrics ####")
    print("\n===="+ text+" SET ====")
    print("MAE: {0:.3f}".format(overal_mae))
    print("RMSE: {0:.3f}".format(overal_rmse))
    print("MAEs: ", np.round(MAEs,3))
    print("RMSEs: ", np.round(RMSEs,3))

    if plot:
        plt.plot(np.arange(len(RMSEs)), RMSEs, label=True)
        plt.plot(np.arange(len(MAEs)), MAEs, label=True)
        plt.grid(linestyle="--")
        plt.xlabel("Community number")
        plt.legend(["RMSE", "MAE"])
        plt.title("Performance metrics for "+ text +" dataset")
        plt.show()

    return overal_mae, MAEs, overal_rmse, RMSEs